In [ ]:
from preprocess import get_data

import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [3]:
data = get_data()

In [4]:
X = data["values"]
X

array([[ -56.,  -50.,  -64., ...,  -84.,  -91.,  -96.],
       [ -91.,  -78.,  -75., ...,  -17.,  -27.,  -42.],
       [ -61.,  -59.,  -46., ..., -120., -111., -118.],
       ...,
       [ 411.,  422.,  401., ...,  136.,  102.,   87.],
       [  53.,   -8.,  -55., ...,  293.,  239.,  180.],
       [  93.,   24.,  -51., ..., -231., -272., -272.]], dtype=float32)

In [5]:
def rename_labels(labels: np.array) -> np.array:
    transformed_labels = np.zeros_like(labels)
    
    # Apply the transformation logic
    transformed_labels[labels == 1] = 2  # seizure
    transformed_labels[(labels == 2) | (labels == 3)] = 1  # risk
    return transformed_labels # other without risk

Y = rename_labels(data["labels"])
Y

array([0, 0, 0, ..., 2, 2, 2], dtype=int32)

In [6]:
# Split for train and test data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Scale and normalize data
scaler =  StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled =  scaler.transform(X_test)

In [7]:
svm_model = SVC(kernel='rbf', C=69)
svm_model.fit(X_train_scaled, y_train)

y_pred = svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8999122036874452

In [13]:
def save_model():
    with open('../trained/svm_model.pkl', 'wb') as file:
        pickle.dump(svm_model, file)

    with open("../trained/scaler.pkl", "wb") as file:
        pickle.dump(scaler, file)


save_model()